In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten
import matplotlib.pyplot as plt
from model_branches import build_cnn_branch, build_lstm_branch
from data_preparation import load_and_split_data

def build_combined_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    cnn_output = build_cnn_branch(input_layer)
    cnn_output = Flatten()(cnn_output)
    lstm_output = build_lstm_branch(input_layer)
    merged = Concatenate()([cnn_output, lstm_output])
    final_output = Dense(num_classes, activation='softmax')(merged)
    model = Model(inputs=input_layer, outputs=final_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def plot_accuracy(history):
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    csv_directory = '/content/drive/MyDrive/imuz/autoz/AutomaticExtractionData_IMUZCenter/'  # غيّري المسار حسب موقع البيانات
    X_train, X_test, y_train, y_test, label_encoder = load_and_split_data(csv_directory)
    num_classes = len(label_encoder.classes_)
    
    model = build_combined_model(X_train.shape[1:], num_classes)
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=30,
        batch_size=32
    )

    plot_accuracy(history)
